<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [ ]:
# method most_simmilar

'''your code'''

wv_embeddings.most_similar(positive=["dog"], topn=27)


Ответ: слово cat не входит, но входит слово cats, 4-е место

слово cat на 26 месте

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
import spacy
# import pymorphy2
import pandas as pd
import re
from nltk.tokenize import WordPunctTokenizer
from tqdm.notebook import tqdm
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer

class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)

tokenizer = MyTokenizer()

In [ ]:
# тестирование
tokenizer.tokenize('I live in Moscow')

In [ ]:
# """
#     question: строка
#     embeddings: наше векторное представление
#     dim: размер любого вектора в нашем представлении

#     return: векторное представление для вопроса

'''your code'''

def question_to_vec(question, embeddings, tokenizer, dim=200):

    result = []

    for word in tokenizer.tokenize(question.lower()):
        try:
        # if word in wv_embeddings:
            result.append(embeddings[word])
        except (KeyError):
            continue

    if len(result):
        result = np.mean(result, axis = 0)
    else:
        result = np.zeros(dim)
    return result

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
'''your code'''

q = 'I love neural networks '
vector = [question_to_vec(q, wv_embeddings, tokenizer, 200) for text in q ]
vector[0].shape

In [ ]:
vector = [question_to_vec(q, wv_embeddings, tokenizer, 200) for text in q ]
vector[0].shape

In [ ]:
vector[0][2].round(2)

Ответ: -1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

Hits@47 = 1

DCG@1 = 0

Ответ: 1

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [ ]:
(1/np.log2(1+9)*(9 <= 10)).round(1)

Ответ: 0.3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

Hits@K=1𝑁∑𝑖=1𝑁[𝑟𝑎𝑛𝑘_𝑞′𝑖≤𝐾],

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''

    hits_value = 1/len(dup_ranks)*sum([(rank <= k) for rank in dup_ranks])

    return hits_value

In [ ]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    dcg_value = 1/len(dup_ranks) * sum([(rank <= k) * 1/(np.log2(1 + rank)) for rank in dup_ranks])
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
!unzip stackoverflow_similar_questions.zip

Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.split('\t'))
        '''your code'''
    return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('/content/data/validation.tsv')
# train_data = read_corpus('/content/data/train.tsv')

Кол-во строк

In [ ]:
validation_data[0]

In [ ]:
len(validation_data)

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
'''your code'''

def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    '''your code'''

    pairs = []

    question_v = question_to_vec(question, embeddings, tokenizer, dim)
    # print(question_v)

    for i, el in enumerate(candidates):
        candidate_v =  question_to_vec(el, embeddings, tokenizer, dim)
        cos = cosine_similarity(question_v.reshape(1, -1), candidate_v.reshape(1, -1))
        pairs.append((i, el, cos[0][0]))

    pairs.sort(key=lambda c: c[2], reverse=True)

    return [el[:-1] for el in deepcopy(pairs)]


Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

In [ ]:
hide = ''
for r in ranks:
    print(r)
    hide += str(r[0])
print(hide)

ОТВЕТ: 021

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
# функция для вычисления метрик ранга
def rank(tokenizer, mve):
    wv_ranking = []
    max_validation_examples = mve
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    return wv_ranking

In [ ]:
# альтернативный токенайзер Spacy с нормализацией токенов
nlp = spacy.load('en_core_web_sm')

class SpacyT:
    def __init__(self):
        pass
    def tokenize(self, text):
        spacy_results = nlp(text)
        return [token.lemma_ for token in spacy_results]

In [ ]:
res = {}

tokenizers = { MyTokenizer() : 'MyTokenizer', # разбиение на токены
              SpacyT() : 'Spacy', # применение нормализации токенов
               WordPunctTokenizer() : 'WordPunctTokenizer'} # токенизация с учетом пунктуации

res_df = pd.DataFrame()

for k, v in tokenizers.items():
    wv_ranking = rank(k, 10)
    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        res[f'DCG@:{k}'] = dcg_score(wv_ranking, k)
        res [f'Hits@:{k}'] = hits_count(wv_ranking, k)
    res = dict(sorted(res.items(), key=lambda item: item[0]))
    df = pd.DataFrame.from_dict(res, orient='index', columns = [v])
    res_df = pd.concat((res_df, df), axis = 1)
res_df

Вывод: применение нормализации (в данном случае через Spacy) улучшает результат, но очень сильно замедляет вычисление

### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('./data/train.tsv')

In [ ]:
train_data[:5]

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
'''your code'''
pair = [p[0] + str(' ') + p[1] for p in train_data] # склеивание двух предложений
pair[:5]


In [ ]:
words = [text.lower().split() for text in pair]
words[10]

In [ ]:
from gensim.models import Word2Vec

embeddings_trained = Word2Vec(words, # data for model to train on
                vector_size = 200,                 # embedding vector size
                min_count = 1,             # consider words that occured at least 5 times
                window = 5).wv

In [ ]:
embeddings_trained.most_similar(positive=["python"], topn=5) # тест модели на выводе близких слов

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

In [ ]:
# обернем расчет метрик в функцию, чтобы дальше использовать её в цикле для расчета по разным окнам
def rank_w2v(tokenizer):
    wv_ranking = []
    max_validation_examples = 10
    for i, line in enumerate(validation_data):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    return wv_ranking

In [ ]:
# Пробуем разный размер окна  / cbow / min_count = 5

res_w2v = {}
res_df_w2v = pd.DataFrame()

window = [1, 2, 3, 4, 5]

tokenizer = MyTokenizer()

for w in window:
    embeddings_trained = Word2Vec(words, # data for model to train on
                    vector_size = 200,                 # embedding vector size
                    min_count = 1,             # consider words that occured at least 1 times
                    window = w).wv  # cbow
    wv_ranking = rank_w2v(tokenizer)

    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        res_w2v[f'DCG@:{k}'] = dcg_score(wv_ranking, k)
        res_w2v [f'Hits@:{k}'] = hits_count(wv_ranking, k)
    res = dict(sorted(res_w2v.items(), key=lambda item: item[0]))
    df = pd.DataFrame.from_dict(res, orient='index', columns = [f'window={w}'])
    res_df_w2v = pd.concat((res_df_w2v, df), axis = 1)

res_df_w2v

In [ ]:
# Пробуем разный размер окна / cbow / min_count = 5

res_w2v = {}
res_df_w2v = pd.DataFrame()

window = [1, 2, 3, 4, 5]

tokenizer = MyTokenizer()

for w in window:
    embeddings_trained = Word2Vec(words, # data for model to train on
                    vector_size = 200,                 # embedding vector size
                    min_count = 5,             # consider words that occured at least 5 times
                    window = w).wv  # cbow
    wv_ranking = rank_w2v(tokenizer)

    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        res_w2v[f'DCG@:{k}'] = dcg_score(wv_ranking, k)
        res_w2v [f'Hits@:{k}'] = hits_count(wv_ranking, k)
    res = dict(sorted(res_w2v.items(), key=lambda item: item[0]))
    df = pd.DataFrame.from_dict(res, orient='index', columns = [f'window={w}'])
    res_df_w2v = pd.concat((res_df_w2v, df), axis = 1)

res_df_w2v

In [ ]:
# Пробуем разный размер окна  / skip-gram / min_count = 5

res_w2v = {}
res_df_w2v = pd.DataFrame()

window = [1, 2, 3, 4, 5]

tokenizer = MyTokenizer()

for w in window:
    embeddings_trained = Word2Vec(words, # data for model to train on
                    vector_size = 200,                 # embedding vector size
                    min_count = 5,             # consider words that occured at least 5 times
                    window = w,
                    sg = 1).wv  # skip-gram
    wv_ranking = rank_w2v(tokenizer)

    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        res_w2v[f'DCG@:{k}'] = dcg_score(wv_ranking, k)
        res_w2v [f'Hits@:{k}'] = hits_count(wv_ranking, k)
    res = dict(sorted(res_w2v.items(), key=lambda item: item[0]))
    df = pd.DataFrame.from_dict(res, orient='index', columns = [f'window={w}'])
    res_df_w2v = pd.concat((res_df_w2v, df), axis = 1)

res_df_w2v

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

## Вывод:


Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?

В данном эксперименте лучше всего показал себя токенезатор Spacy с использованием лемм (нормальных форм), НО он работает гораздо медленнее более более порстых токенизаторов и потому проверить его работу удалось на малом клличестве примеров.

Если выбирать между токенизатором с применением регулярных выражений, убирающих понктуацию и токенизатором, учитывающим пунктуацию, то первый показал себя лучше, так как значи препинания в данном случае не сильно влияют на контекст из-за специфики текстов.

* Помогает ли нормализация слов?
Да, нормализация помогает.

* Какие эмбеддинги лучше справляются с задачей и почему?

Лучше всего справились с задачей эмбединги, созданные на основе Skip-Gram, с окном window = 5 и min_count = 5. Окно 5 позволило лучше определять конкстые слова по основному, видимо из-за особенностей текста, в котором ключевые слова встречаются в одном и том же окружении.


* Почему получилось плохое качество решения задачи?

Качество получено не очень хорошее из-за того, что w2v не запоминает контекст, технология устарела и имеет свои минусы.


* Предложите свой подход к решению задачи.

Использовать другие эмбеддинги, например, Glove, которые можно строить на n-граммах, что позволит лучше учитывать словоформы и соответственно контекст.

